# RNA Expression Matrix Preprocessing

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import scrublet as scr
import re
import os

In [2]:
def list_directories(path):
    # List all items in the directory
    items = os.listdir(path)
    # Filter and return only directories
    directories = [item for item in items if os.path.isdir(os.path.join(path, item))]
    return directories

# Specify the path
path = "/cndd/xic007/scorch/SCORCH_NHP/downloaded_data"

# Call the function and print the directory names
directory_names = list_directories(path)
print(directory_names)

['JB_1415_1_2_JB_1411_1_2', 'JD_173_1_2_JD_169_1_2', 'JD_174_1_2_JD_170_1_2', 'JB_1414_1_2_JB_1410_1_2', 'JB_1416_1_2_JB_1412_1_2', 'JB_1417_1_2_JB_1413_1_2', 'JD_175_1_2_JD_171_1_2']


In [3]:
meta = pd.read_csv("/cndd/xic007/scorch/SCORCH_NHP/Rana_SCORCH_NHP_SIV-NeMO submission - Sheet C.2 - Macaque Data Production Tracking.csv", header=None)
meta.columns = meta.iloc[1]  # Set the first row as header 
meta = meta[2:]
meta

1,10X Run ID,Sample ID,Donor ID,Sample description,"Disease group [Control, HIV, OUD, HIV+OUD]",Brain region,OBSOLETE Tissue source,Batch,RA,Planned experimental date,...,Fastq (ATAC),Fastq (RNA),Fastq deep (ATAC),Fastq deep (RNA),deep+shallow ATAC Median high-quality fragments per cell,deep+shallow ATAC TSS enrichment score,deep+shallow RNA median genes per cell,deep+shallow clustering qual,comment to average/bas samples,Comments
2,SCORCH_MULTI_24,M108/NM25 Amygdala,NaN,NaN,Control,NaN,NaN,1,JD,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SCORCH_MULTI_24,M108/NM25 Hippocampus,NaN,NaN,Control,NaN,NaN,1,JD,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SCORCH_MULTI_24,M108/NM25 Prefrontal Cortex,NaN,NaN,Control,NaN,NaN,1,JD,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SCORCH_MULTI_24,M108/NM25 Striatum,NaN,NaN,Control,NaN,NaN,1,JD,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SCORCH_Macaque_2,M108/NN17 Amygdala,NaN,NaN,Control,NaN,NaN,2,JB,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SCORCH_Macaque_2,MM108/NN17 Hippocampus,NaN,NaN,Control,NaN,NaN,2,JB,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SCORCH_Macaque_2,M108/NN17 PFC,NaN,NaN,Control,NaN,NaN,2,JB,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SCORCH_Macaque_2,M108/NN17 Striatum,NaN,NaN,Control,NaN,NaN,2,JB,NaN,...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,https://epigenomics.sdsc.edu/hjiao/bot_output/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Extract relevant columns
sample_ids = meta["Sample ID"].tolist()
result_ids = meta["Library ID (RNA)"].tolist()  # Adjust the column name if needed

# Create a dictionary mapping "10X Run ID" to "Sample ID"
result_to_sample_mapping = {result: sample for result, sample in zip(result_ids, sample_ids)}

# Convert to the desired structure
# Merge "PFC" and "Prefrontal Cortex" into "Prefrontal Cortex"
library_dict = {
    library: {
        'sample_id': value.split()[0],  # Extract sample ID
        'region': "PFC" if "PFC" in value or "Prefrontal Cortex" in value else " ".join(value.split()[1:])  # Standardize region name
    }
    for library, value in result_to_sample_mapping.items()
}

# Display the updated dictionary
library_dict

{'JD_173': {'sample_id': 'M108/NM25', 'region': 'Amygdala'},
 'JD_174': {'sample_id': 'M108/NM25', 'region': 'Hippocampus'},
 'JD_175': {'sample_id': 'M108/NM25', 'region': 'PFC'},
 'JD_176': {'sample_id': 'M108/NM25', 'region': 'Striatum'},
 'JB_1414': {'sample_id': 'M108/NN17', 'region': 'Amygdala'},
 'JB_1415': {'sample_id': 'MM108/NN17', 'region': 'Hippocampus'},
 'JB_1416': {'sample_id': 'M108/NN17', 'region': 'PFC'},
 'JB_1417': {'sample_id': 'M108/NN17', 'region': 'Striatum'}}

In [5]:
# Regular expression to identify mitochondrial genes
mito_pattern = r'^ND[1-6L]$|^CYTB$|^COX[1-3]$|^ATP[68]$|^MT-'

for directory in directory_names:
    library = re.match(r'^([^_]+_\d+)_1_2', directory).group(1)

    sample_id = library_dict[library]['sample_id']
    region = library_dict[library]['region']
    
    # Load the data
    data = sc.read_10x_h5(f"/cndd/xic007/scorch/SCORCH_NHP/downloaded_data/{directory}/outs/filtered_feature_bc_matrix.h5")

    data.var_names_make_unique()
    
    # Add metadata to observations
    data.obs['Sample_id'] = sample_id
    data.obs['Condition'] = 'Control'
    data.obs['Brain Region'] = region

    data.obs_names = [f"{sample_id}_{region}_{idx}" for idx in data.obs_names]

    print(f'Filter complete. Filtered data has {data.n_obs} cells, {data.n_vars} genes.')
    
    # Perform Scrublet analysis for doublet detection
    scrub = scr.Scrublet(data.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()

    data.obs['Doublet Score'] = doublet_scores
    data.obs['Predicted Doublet'] = predicted_doublets

    print('Predicted doublets marked.')
    
    # Identify mitochondrial genes
    data.var["mt"] = data.var.index.str.contains(mito_pattern, na=False)
    
    # Identify ribosomal genes
    data.var["ribo"] = data.var.index.str.startswith(("RPS", "RPL"))

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(data, qc_vars=["mt", "ribo"], inplace=True)

    print('QC metrics calculation complete.')

    # Calculate percent mitochondrial content
    data.obs['percent_mito'] = np.sum(data[:, data.var["mt"]].X, axis=1).A1 / np.sum(data.X, axis=1).A1

    print(f'Number of mito genes: {data.var["mt"].sum()}.')

    # Calculate percent ribosomal content
    print(f'Number of ribo genes: {data.var["ribo"].sum()}.')

    data.obs['percent_ribo'] = np.sum(data[:, data.var["ribo"]].X, axis=1).A1 / np.sum(data.X, axis=1).A1

    # Visualize QC metrics
    sc.pl.violin(data, ['percent_mito', 'percent_ribo'], stripplot=False, jitter=0.4)

    # Identify genes to remove (mitochondrial and MALAT1)
    malat1_genes = data.var_names.str.contains('^MALAT1$', na=False)
    genes_to_remove = np.logical_or(data.var["mt"], malat1_genes)
    data = data[:, ~genes_to_remove]

    print(f'Mito genes & MALAT1 removed. Data now has {data.n_obs} cells, {data.n_vars} genes.')

    # Store raw data in a separate layer
    data.layers['RNA_raw'] = data.X.copy()
    
    # Save the preprocessed data
    data.write_h5ad(f'/cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/{directory}.h5ad')

    print(f'Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/{directory}.h5ad')

print("All processes done.")

/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Filter complete. Filtered data has 14258 cells, 21369 genes.
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.81
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 1.7%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 3.8%
Elapsed time: 32.5 seconds
Predicted doublets marked.
QC metrics calculation complete.
Number of mito genes: 12.
Number of ribo genes: 81.
Mito genes & MALAT1 removed. Data now has 14258 cells, 21357 genes.


/tmp/ipykernel_1394908/362043245.py:65: ImplicitModificationWarning: Setting element `.layers['RNA_raw']` of view, initializing view as actual.
  data.layers['RNA_raw'] = data.X.copy()


Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/JB_1415_1_2_JB_1411_1_2.h5ad


/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/JB_1417_1_2_JB_1413_1_2.h5ad


/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Filter complete. Filtered data has 20000 cells, 21369 genes.
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.14
Detected doublet rate = 24.1%
Estimated detectable doublet fraction = 76.1%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 31.7%
Elapsed time: 47.1 seconds
Predicted doublets marked.
QC metrics calculation complete.
Number of mito genes: 12.
Number of ribo genes: 81.
Mito genes & MALAT1 removed. Data now has 20000 cells, 21357 genes.


/tmp/ipykernel_1394908/362043245.py:65: ImplicitModificationWarning: Setting element `.layers['RNA_raw']` of view, initializing view as actual.
  data.layers['RNA_raw'] = data.X.copy()


Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/JD_175_1_2_JD_171_1_2.h5ad
All processes done.


In [7]:
additional = ['/datasets/Public_Datasets/Rana_SCORCH_NHP_SIV/counts/QY_1686_1_2_3_QY_1685_1_2_3',
              '/datasets/Public_Datasets/Rana_SCORCH_NHP_SIV/counts/QY_1688_1_2_3_QY_1687_1_2_3',
              '/datasets/Public_Datasets/Rana_SCORCH_NHP_SIV/counts/QY_1690_1_2_3_QY_1689_1_2_3']
sample_names = ['Rana_NHP_40495',
               'Rana_NHP_40965',
               'Rana_NHP_41056']
directory = ['QY_1686_1_2_3_QY_1685_1_2_3',
            'QY_1688_1_2_3_QY_1687_1_2_3',
            'QY_1690_1_2_3_QY_1689_1_2_3']

In [8]:
# Regular expression to identify mitochondrial genes
mito_pattern = r'^ND[1-6L]$|^CYTB$|^COX[1-3]$|^ATP[68]$|^MT-'

for directory, sample, save_directory in zip(additional, sample_names, directory):

    sample_id = sample
    region = 'PFC'
    
    # Load the data
    data = sc.read_10x_h5(f"{directory}/outs/filtered_feature_bc_matrix.h5")

    data.var_names_make_unique()
    
    # Add metadata to observations
    data.obs['Sample_id'] = sample_id
    data.obs['Condition'] = 'Control'
    data.obs['Brain Region'] = region

    data.obs_names = [f"{sample_id}_{region}_{idx}" for idx in data.obs_names]

    print(f'Filter complete. Filtered data has {data.n_obs} cells, {data.n_vars} genes.')
    
    # Perform Scrublet analysis for doublet detection
    scrub = scr.Scrublet(data.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()

    data.obs['Doublet Score'] = doublet_scores
    data.obs['Predicted Doublet'] = predicted_doublets

    print('Predicted doublets marked.')
    
    # Identify mitochondrial genes
    data.var["mt"] = data.var.index.str.contains(mito_pattern, na=False)
    
    # Identify ribosomal genes
    data.var["ribo"] = data.var.index.str.startswith(("RPS", "RPL"))

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(data, qc_vars=["mt", "ribo"], inplace=True)

    print('QC metrics calculation complete.')

    # Calculate percent mitochondrial content
    data.obs['percent_mito'] = np.sum(data[:, data.var["mt"]].X, axis=1).A1 / np.sum(data.X, axis=1).A1

    print(f'Number of mito genes: {data.var["mt"].sum()}.')

    # Calculate percent ribosomal content
    print(f'Number of ribo genes: {data.var["ribo"].sum()}.')

    data.obs['percent_ribo'] = np.sum(data[:, data.var["ribo"]].X, axis=1).A1 / np.sum(data.X, axis=1).A1

    # Visualize QC metrics
    sc.pl.violin(data, ['percent_mito', 'percent_ribo'], stripplot=False, jitter=0.4)

    # Identify genes to remove (mitochondrial and MALAT1)
    malat1_genes = data.var_names.str.contains('^MALAT1$', na=False)
    genes_to_remove = np.logical_or(data.var["mt"], malat1_genes)
    data = data[:, ~genes_to_remove]

    print(f'Mito genes & MALAT1 removed. Data now has {data.n_obs} cells, {data.n_vars} genes.')

    # Store raw data in a separate layer
    data.layers['RNA_raw'] = data.X.copy()
    
    # Save the preprocessed data
    data.write_h5ad(f'/cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/{save_directory}.h5ad')

    print(f'Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/{save_directory}.h5ad')

print("All processes done.")

/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Filter complete. Filtered data has 11743 cells, 21369 genes.
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.63
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 6.6%
Elapsed time: 26.2 seconds
Predicted doublets marked.
QC metrics calculation complete.
Number of mito genes: 12.
Number of ribo genes: 81.
Mito genes & MALAT1 removed. Data now has 11743 cells, 21357 genes.


/tmp/ipykernel_2344991/2323987043.py:64: ImplicitModificationWarning: Setting element `.layers['RNA_raw']` of view, initializing view as actual.
  data.layers['RNA_raw'] = data.X.copy()


Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/QY_1686_1_2_3_QY_1685_1_2_3.h5ad


/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/QY_1688_1_2_3_QY_1687_1_2_3.h5ad


/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/AD/xic007/miniforge3/envs/scorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Filter complete. Filtered data has 11607 cells, 21369 genes.
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.72
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 9.5%
Elapsed time: 25.3 seconds
Predicted doublets marked.
QC metrics calculation complete.
Number of mito genes: 12.
Number of ribo genes: 81.
Mito genes & MALAT1 removed. Data now has 11607 cells, 21357 genes.


/tmp/ipykernel_2344991/2323987043.py:64: ImplicitModificationWarning: Setting element `.layers['RNA_raw']` of view, initializing view as actual.
  data.layers['RNA_raw'] = data.X.copy()


Data preprocessed and saved to /cndd/xic007/scorch/SCORCH_NHP/Processed_GEX/QY_1690_1_2_3_QY_1689_1_2_3.h5ad
All processes done.
